In [1]:
# https://medium.com/@jebaseelanravi96/machine-learning-iris-classification-33aa18a4a983


import numpy as np
import pandas as pd

input_file = "iris.csv"

iris_df = pd.read_csv(input_file, header = 0)
iris_df.head()

# I will likely pollute the data a bit and have them do some kind of data cleaning, right now its clean though
# I also want to introduce a "categorical" feature like color, just to force them to 
# encode it / do a tiny bit of data transforming
iris_df.isnull()
iris_df = iris_df.dropna()
iris_df = pd.get_dummies(iris_df, columns=["color"])

# feature_cols = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm','color_green','color_pink','color_purple','color_orange','color_red','color_black','color_white']
feature_cols = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm']



x = iris_df.loc[:, feature_cols]
y = iris_df.Species
#iris_df.head()


In [2]:
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:


from sklearn.model_selection import GridSearchCV,train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.5)

from sklearn import tree


classifier=tree.DecisionTreeClassifier()


classifier.fit(x_train,y_train)
predictions=classifier.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

0.9333333333333333


In [4]:
# Now let's encourage an optimization workflow here, somehow can we coach them to track this via jobs?
# I found some pointers on optimizing a decision tree here 
# https://ai.plainenglish.io/hyperparameter-tuning-of-decision-tree-classifier-using-gridsearchcv-2a6ebcaffeda



param_dict = {
    "criterion":['gini','entropy'],
    "max_depth":range(1,10),
    "min_samples_split":range(2,10),
    "min_samples_leaf":range(1,5),
}
# (imported  GridSearchCV already in previous cell)
grid = GridSearchCV(classifier,
                       param_grid=param_dict,
                       cv=10,
                       verbose=1,
                       n_jobs=-1)

# note this kind of seems like its working but then there's an error? I can't tell.
grid.fit(x_train,y_train)

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Done 340 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:   15.4s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': range(1, 10), 'min_samples_split': range(2, 10), 'min_samples_leaf': range(1, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [5]:
#print the best parameters
grid.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 3}

In [6]:
#new accuracy score after grid search
predictions_grid = grid.predict(x_test)
print(accuracy_score(y_test,predictions_grid))

0.9466666666666667


In [7]:
#fit model on entire dataset
grid.fit(x,y)

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Done 520 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 4720 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:   16.2s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': range(1, 10), 'min_samples_split': range(2, 10), 'min_samples_leaf': range(1, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [10]:
#export the trained model as a pickle file for later use
import pickle
with open("../artifacts/model.pkl", "wb") as f:
      pickle.dump(grid, f)
with open("model.pkl", "wb") as f:
      pickle.dump(grid, f)